<font size="6">Run the LULUCF part of the AFOLU model</font> 

<font size="4">Must be run using the utilities_and_variables.ipynb kernel</font> 

In [129]:
# Function to calculate LULUCF fluxes and carbon densities
# Operates pixel by pixel, so uses numba (Python compiled to C++).
@jit(nopython=True)
def LULUCF_fluxes(uint8_layers, float32_layers):

    # print("Inside numba")
    # print(uint8_layers)
    # print(uint8_layers["land_cover_2000"])
    # print(uint8_layers["land_cover_2000"].shape)

    # Separate dictionaries for output numpy arrays of each datatype, named by output type (e.g., AGC density, BGC removals) and year ([output_type}_[year]).
    # This is because a dictionary in a Numba function cannot have arrays with multiple data types, so each dictionary has to store only one data type,
    # just like inputs to the function.
    out_dict_uint16 = {}
    out_dict_float32 = {}

    end_years = list(range(first_year, last_year+1, 5))[1:]

    for year in end_years:
                
        # print(year)
        state_out = np.zeros(uint8_layers["land_cover_2000"].shape).astype('uint16')  
        agc_dens_out = np.zeros(float32_layers["agc_2000"].shape).astype('float32')
        bgc_dens_out = np.zeros(float32_layers["agc_2000"].shape).astype('float32')
        agc_flux_out = np.zeros(float32_layers["agc_2000"].shape).astype('float32')
        bgc_flux_out = np.zeros(float32_layers["agc_2000"].shape).astype('float32')

        LC_previous_block = uint8_layers[f"land_cover_{year-5}"]
        LC_current_block = uint8_layers[f"land_cover_{year}"]
        veg_h_previous_block = uint8_layers[f"vegetation_height_{year-5}"]
        veg_h_current_block = uint8_layers[f"vegetation_height_{year}"]
        planted_forest_type_block = uint8_layers["planted_forest_type"]

        # Iterates through all pixels in the chunk
        for row in range(LC_current_block.shape[0]):
            for col in range(LC_current_block.shape[1]):
                
                LC_previous = LC_previous_block[row, col]
                LC_current = LC_current_block[row, col]
                veg_h_previous = veg_h_previous_block[row, col]
                veg_h_current = veg_h_current_block[row, col]
                planted_forest = planted_forest_type_block[row, col]

                tree_previous = (veg_h_previous >=5)
                tree_current = (veg_h_current >= 5)
                LC_previous_tall_veg = (((LC_previous >= 27) and (LC_previous <= 48)) or ((LC_previous >= 127) and (LC_previous <= 148)))
                LC_current_tall_veg = (((LC_current >= 27) and (LC_current <= 48)) or ((LC_current >= 127) and (LC_current <= 148)))
    
                if (not tree_previous) and (tree_current):           # Non-tree converted to tree     ##TODO: Include mangrove exception.
                    if planted_forest == 0:                          # New non-SDPT trees
                        if LC_current_tall_veg:                      # New trees outside forests
                            state_out[row, col] = 111
                            agc_dens_out[row, col] = 0.5
                        else:                                        # New terrestrial natural forest
                            state_out[row, col] = 112
                            agc_dens_out[row, col] = 11.2
                    else:                                            # New SDPT trees
                        state_out[row, col] = 12 
                        agc_dens_out[row, col] = 55.4

                # if (tree_previous) and (not tree_current):           # Tree converted to non-tree     ##TODO: Include mangrove exception.
                #     state_out[row, col] = 2
                #     if planted_forest == 0:                          # Full loss of non-SDPT trees
                    
                else:                                                # Not covered in above branches
                    state_out[row, col] = 9999
                    agc_dens_out[row, col] = 99.9

        # Adds the output arrays to the dictionary with the appropriate data type
        out_dict_uint16[f"state_{year}"] = state_out
        out_dict_float32[f"agc_dens_{year}"] = agc_dens_out
        # out_dict_float32[f"bgc_dens_{year}"] = bgc_dens_out
        # out_dict_float32[f"agc_flux_{year}"] = agc_flux_out
        # out_dict_float32[f"bgc_flux_{year}"] = bgc_flux_out

    return out_dict_uint16, out_dict_float32

In [146]:
# 
def calculate_and_upload_LULUCF_fluxes(bounds, is_final):

    bounds_str = boundstr(bounds)    # String form of chunk bounds
    tile_id = xy_to_tile_id(bounds[0], bounds[3])    # tile_id in YYN/S_XXXE/W
    chunk_length_pixels = calc_chunk_length_pixels(bounds)   # Chunk length in pixels (as opposed to decimal degrees)

    no_data_val = 255

    
    ### Part 1: Downloads chunks and check for data

    # Dictionary of downloaded layers
    download_dict = {}
    layers = {}

    download_dict = {
        "land_cover_2000": f"{LC_uri}/composite/2000/raw/{tile_id}.tif",
        "land_cover_2005": f"{LC_uri}/composite/2005/raw/{tile_id}.tif",
        "land_cover_2010": f"{LC_uri}/composite/2010/raw/{tile_id}.tif",
        "land_cover_2015": f"{LC_uri}/composite/2015/raw/{tile_id}.tif",
        "land_cover_2020": f"{LC_uri}/composite/2020/raw/{tile_id}.tif",  

        "vegetation_height_2000": f"{LC_uri}/vegetation_height/2000/{tile_id}_vegetation_height_2000.tif",
        "vegetation_height_2005": f"{LC_uri}/vegetation_height/2005/{tile_id}_vegetation_height_2005.tif",
        "vegetation_height_2010": f"{LC_uri}/vegetation_height/2010/{tile_id}_vegetation_height_2010.tif",
        "vegetation_height_2015": f"{LC_uri}/vegetation_height/2015/{tile_id}_vegetation_height_2015.tif",
        "vegetation_height_2020": f"{LC_uri}/vegetation_height/2020/{tile_id}_vegetation_height_2020.tif",  

        "agc_2000": f"s3://gfw2-data/climate/carbon_model/carbon_pools/aboveground_carbon/extent_2000/standard/20230222/{tile_id}_Mg_AGC_ha_2000.tif",
        # "bgc_2000": f"s3://gfw2-data/climate/carbon_model/carbon_pools/belowground_carbon/extent_2000/standard/20230222/{tile_id}_Mg_BGC_ha_2000.tif",
        # "deadwood_c_2000": f"s3://gfw2-data/climate/carbon_model/carbon_pools/deadwood_carbon/extent_2000/standard/20230222/{tile_id}_Mg_deadwood_C_ha_2000.tif",
        # "litter_c_2000": f"s3://gfw2-data/climate/carbon_model/carbon_pools/litter_carbon/extent_2000/standard/20230222/{tile_id}_Mg_litter_C_ha_2000.tif",
        # "soil_c_2000": f"s3://gfw2-data/climate/carbon_model/carbon_pools/soil_carbon/intermediate_full_extent/standard/20231108/{tile_id}_soil_C_full_extent_2000_Mg_C_ha.tif",

        # "r_s_ratio": f"s3://gfw2-data/climate/carbon_model/BGB_AGB_ratio/processed/20230216/{tile_id}_BGB_AGB_ratio.tif",

        # "drivers": f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/tree_cover_loss_drivers/processed/drivers_2022/20230407/{tile_id}_tree_cover_loss_driver_processed.tif",
        "planted_forest_type": f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/plantation_type/SDPTv2/20230911/{tile_id}_plantation_type_oilpalm_woodfiber_other.tif",
        # "peat": f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/peatlands/processed/20230315/{tile_id}_peat_mask_processed.tif",
        # "ecozone": f"s3://gfw2-data/fao_ecozones/v2000/raster/epsg-4326/10/40000/class/gdal-geotiff/{tile_id}.tif",   # Originally from gfw-data-lake, so it's in 400x400 windows 
        # "iso": f"s3://gfw2-data/gadm_administrative_boundaries/v3.6/raster/epsg-4326/10/40000/adm0/gdal-geotiff/{tile_id}.tif",  # Originally from gfw-data-lake, so it's in 400x400 windows
        # "ifl_primary": f"s3://gfw2-data/climate/carbon_model/ifl_primary_merged/processed/20200724/{tile_id}_ifl_2000_primary_2001_merged.tif"
    }

    # for year in range(first_year, last_year+1):     # Annual burned area maps start in 2000
        # download_dict[f"burned_area_{year}"] = f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/burn_year/burn_year_10x10_clip/ba_{year}_{tile_id}.tif",    

    for year in range(first_year+1, last_year):     # Annual forest disturbance maps start in 2001
        download_dict[f"forest_disturbance_{year}"] = f"{LC_uri}/annual_forest_disturbance/raw/{year}_{tile_id}.tif"     

    # Checks whether tile exists at all. Doesn't try to download chunk if the tile doesn't exist.
    tile_exists = check_for_tile(download_dict)

    if tile_exists == 0:
        return

    # Note: If running in a local Dask cluster, prints to console may be duplicated. Doesn't happen with a Coiled cluster of the same size (1 worker).
    # Seems to be a problem with local Dask getting overwhelmed by so many futures being created and downloaded from s3. 
    futures = prepare_to_download_chunk(bounds, download_dict, no_data_val)

    # print(futures)

    if not is_final:
        print(f"Waiting for requests for data in chunk {bounds_str} in {tile_id}: {timestr()}")
    
    # Waits for requests to come back with data from S3
    for future in concurrent.futures.as_completed(futures):
        layer = futures[future]
        layers[layer] = future.result()

    # print(layers)
    
    # Checks chunk for data. Skips the chunk if it has no data in it.
    data_in_chunk = check_chunk_for_data(layers, "land_cover_", bounds_str, tile_id, no_data_val)

    if data_in_chunk == 0:
        return

    
    ### Part 2: Create a separate dictionary for each chunk datatype so that they can be passed to Numba as separate arguments.
    ### Numba functions can accept (and return) dictionaries of arrays as long as each dictionary only has arrays of one data type (e.g., uint8, float32)
    ### Note: need to add new code if inputs with other data types are added

    # Initializes empty dictionaries for each type
    uint8_dict_layers = {}
    float32_dict_layers = {}
    
    # Iterates through the downloaded chunk dictionary and distributes arrays to a separate dictionary for each data type
    for key, array in layers.items():
        if array.dtype == np.uint8:
            uint8_dict_layers[key] = array
        elif array.dtype == np.float32:
            float32_dict_layers[key] = array
        else:
            raise TypeError(f"{key} dtype not in list")

    print(f"uint8 datasets: {uint8_dict_layers.keys()}")
    print(f"float32 datasets: {float32_dict_layers.keys()}")
    
    # Creates numba-compliant typed dict for each type of array
    typed_dict_uint8 = Dict.empty(
        key_type=types.unicode_type, 
        value_type=types.Array(types.uint8, 2, 'C')  # Assuming 2D arrays of uint8
    )

    typed_dict_float32 = Dict.empty(
        key_type=types.unicode_type, 
        value_type=types.Array(types.float32, 2, 'C')  # Assuming 2D arrays of float32
    )

    # Populates the numba-compliant typed dicts
    for key, array in uint8_dict_layers.items():
        typed_dict_uint8[key] = array

    for key, array in float32_dict_layers.items():
        typed_dict_float32[key] = array
        
    ### Part 3: Calculates LULUCF fluxes and densities
   
    print(f"Calculating LULUCF fluxes and carbon densities in {bounds_str} in {tile_id}: {timestr()}")

    out_dict_uint16, out_dict_float32 = LULUCF_fluxes(
        typed_dict_uint8, typed_dict_float32 
    )

    print(out_dict_uint16)
    print(out_dict_float32)
    print(f"Average of {list(out_dict_uint16.keys())[0]} is: {list(out_dict_uint16.values())[0].mean()}")

    ### Part 4: Save small rasters and upload to s3
    
    # # Output files to upload to s3
    # LULUCF_output_dict[f"IPCC_change_{year-5}_{year}"] = [IPCC_change, "uint8", "IPCC_basic_change", f'{year-5}_{year}']  

    save_and_upload_small_raster_set(bounds, chunk_length_pixels, tile_id, bounds_str, out_dict_uint16, is_final)

    
    # Clear memory of unneeded arrays
    del out_dict_uint16
    del out_dict_float32

    return f"Success for {bounds_str}: {timestr()}"

In [145]:
%%time

## Create LULUCF flux and carbon stock 2x2 deg rasters 

## Area to analyze
## chunk_params arguments: W, S, E, N, chunk size (degrees)
# chunk_params = [-180, -60, 180, 80, 2]  # entire world
# chunk_params = [-10, 40, 20, 70, 1]    # 30x30 deg (70N_010W), 900 chunks
# chunk_params = [10, 40, 20, 50, 2]    # 10x10 deg (50N_010E), 25 chunks
# chunk_params = [10, 40, 20, 50, 10]    # 10x10 deg (50N_010E), 1 chunk
# chunk_params = [10, 46, 14, 50, 2]   # 4x4 deg, 4 chunks
# chunk_params = [110, -10, 114, -6, 2]   # 4x4 deg, 4 chunks
# chunk_params = [10, 48, 12, 50, 1]   # 2x2 deg, 4 chunks
# chunk_params = [10, 49, 11, 50, 1]   # 1x1 deg, 1 chunk
# chunk_params = [10, 49, 11, 50, 0.5] # 1x1 deg, 4 chunks
# chunk_params = [10, 49.5, 10.5, 50, 0.25] # 0.5x0.5 deg, 4 chunks
# chunk_params = [10, 42, 11, 43, 0.5] # 1x1 deg, 4 chunks (some GLCLU code=254 for ocean and some land, so data should be output)
chunk_params = [10, 49.75, 10.25, 50, 0.25] # 0.25x0.25 deg, 1 chunk (has data)

# # Range of no-data cases for testing
# chunk_params = [110, -10, 120, 0, 2]    # 10x10 deg (00N_110E), 25 chunks (all chunks have land and should be output)
# chunk_params = [110, -20, 120, -10, 2]    # 10x10 deg (00N_110E), 25 chunks (all chunks have land and should be output)
# chunk_params = [0, 79.75, 0.25, 80, 0.25] # 0.25x0.25 deg, 1 chunk (no 80N_000E tile-- no data)
# chunk_params = [112, -12, 116, -8, 2]   # 2x2 deg, 1 chunk (bottom of Java, has data but mostly ocean)
# chunk_params = [10.875, 41.75, 11, 42, 0.25] # 0.25x0.25 deg, 1 chunk (entirely GLCLU code=255 for ocean, so no actual data-- nothing should be be output)
# chunk_params = [-10, 21.75, -9.75, 22, 0.25] # 0.25x0.25 deg, 1 chunk (has data but entirely desert (fully GLCLU code=0))
# chunk_params = [10, 49.75, 10.25, 50, 0.25] # 0.25x0.25 deg, 1 chunk (has data)


# Makes list of chunks to analyze
chunks = get_chunk_bounds(chunk_params)  
print("Processing", len(chunks), "chunks")
# print(chunks)

# Determines if the output file names for final versions of outputs should be used
is_final = False
if len(chunks) > 30:
    is_final = True
    print("Running as final model.")

# Creates list of tasks to run (1 task = 1 chunk for all years)
delayed_result = [dask.delayed(calculate_and_upload_LULUCF_fluxes)(chunk, is_final) for chunk in chunks]

# Actually runs analysis
results = dask.compute(*delayed_result)
results

Processing 1 chunks
Tile id 50N_010E exists. Proceeding.
Requesting data in chunk 10_50_10_50 in 50N_010E: 20240411_17_53_21
Waiting for requests for data in chunk 10_50_10_50 in 50N_010E: 20240411_17_53_32
Data in chunk 10_50_10_50. Proceeding.
uint8 datasets: dict_keys(['forest_disturbance_2001', 'land_cover_2020', 'forest_disturbance_2017', 'forest_disturbance_2019', 'vegetation_height_2015', 'forest_disturbance_2005', 'vegetation_height_2005', 'forest_disturbance_2006', 'planted_forest_type', 'forest_disturbance_2015', 'forest_disturbance_2016', 'vegetation_height_2010', 'forest_disturbance_2007', 'vegetation_height_2020', 'forest_disturbance_2002', 'forest_disturbance_2003', 'forest_disturbance_2018', 'land_cover_2005', 'forest_disturbance_2012', 'vegetation_height_2000', 'forest_disturbance_2004', 'forest_disturbance_2010', 'forest_disturbance_2013', 'forest_disturbance_2009', 'land_cover_2015', 'land_cover_2000', 'land_cover_2010', 'forest_disturbance_2014', 'forest_disturbance_

('Success for 10_50_10_50: 20240411_17_53_32',)

In [ ]:
# Execute without Dask
calculate_and_upload_LULUCF_fluxes([10, 49.75, 10.25, 50], is_final=False)